In [17]:
import requests
import pathlib
import time
from requests_html import HTML
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

In [18]:
BASE_DIR = pathlib.Path.cwd()
DATA_DIR = BASE_DIR / "data"
if not DATA_DIR.exists():
    DATA_DIR.mkdir(exist_ok=True)

product_category_links_output = DATA_DIR / "category-products.csv"
product_output = DATA_DIR / "product.csv"

In [19]:
options = Options()
options.add_argument('--headless')

driver = webdriver.Firefox(options=options, executable_path="./geckodriver")

In [20]:
categories = [
    {"name": "electronics", "url": "https://www.amazon.in/gp/bestsellers/electronics/ref=zg_bs_nav_0"},
    {"name": "computers", "url": "https://www.amazon.in/gp/bestsellers/computers/ref=zg_bs_nav_0"},
    {"name": "accessories", "url": "https://www.amazon.in/gp/bestsellers/apparel/ref=zg_bs_nav_0"},
]
categories

[{'name': 'electronics',
  'url': 'https://www.amazon.in/gp/bestsellers/electronics/ref=zg_bs_nav_0'},
 {'name': 'computers',
  'url': 'https://www.amazon.in/gp/bestsellers/computers/ref=zg_bs_nav_0'},
 {'name': 'accessories',
  'url': 'https://www.amazon.in/gp/bestsellers/apparel/ref=zg_bs_nav_0'}]

In [64]:
import re
regex_options = [
    r"https://amazon.in/gp/product/(?P<product_id>[\w-]+)/",
    r"https://amazon.in/dp/(?P<product_id>[\w-]+)/",
    r"https://amazon.in/(?P<slug>[\w-]+)/dp/(?P<product_id>[\w-]+)/"
]
def extract_productid_from_url(url):
    product_id = None
    for regex_str in regex_options:
        regex = re.compile(regex_str)
        if (match := regex.match(url)) is not None:
            print(match)
            try:
                product_id = match['product_id']
            except:
                pass
    return product_id 

In [65]:
def clean_page_links(page_links=[], category=None):
    final_page_links = []
    for url in page_links:
        product_id = extract_productid_from_url(url)
        # print(product_id)
        if product_id is not None:
            final_page_links.append({"url": url, "product_id": product_id, "category": category})
    return final_page_links

In [66]:
def scrape_category_product_links(categories=[]):
    all_product_links = []
    for category in categories:
        time.sleep(1.5)
        url = category.get("url")
        driver.get(url)
        body_el = driver.find_element_by_css_selector('body')
        html_str = body_el.get_attribute("innerHTML")
        html_obj = HTML(html=html_str)
        page_links = [f"https://www.amazon.in{x}" for x in html_obj.links if x.startswith("/")]
        cleaned_page_links = clean_page_links(page_links, category)
        # print(cleaned_page_links)
        all_product_links += cleaned_page_links
    return all_product_links


In [67]:
all_path_links = scrape_category_product_links(categories=categories)
# all_path_links

In [58]:
all_path_links

[]

In [9]:
import pandas as pd
category_df = pd.DataFrame(all_product_links)
category_df

NameError: name 'all_product_links' is not defined

In [10]:
category_df.to_csv(product_category_links_output, index=False)

NameError: name 'category_df' is not defined

In [14]:
def scrape_product_page(url, title_lookup="#productTitle",price_lookup="#priceblock_ourprice"):
    driver.get(url)
    time.sleep(1.5)
    body_el = driver.find_element_by_css_selector("body")
    html_str = body_el.get_attribute("innerHTML")
    html_obj = HTML(html=html_str)
    product_title = html_obj.find(title_lookup, first=True).text
    product_price = html_obj.find(price_lookup, first=True).text
    return product_title, product_price

In [16]:
def row_scrape_event(row, *args, **kwargs):
    link = row['url']
    scraped = 0
    try:
        scraped = row['scraped']
    except:
        pass
    if scraped ==1 or scraped =="1":
        return row
    product_id = row['product_id']
    title, price = None, None
    try:
        title, price = scrape_product_page(link)
    except:
        pass
    row['title'] = title
    row['price'] = price
    row['timestamp'] = datetime.datetime.now().timestamp()
    row['scraped'] = 1
    return row


In [12]:
df = pd.read_csv(product_category_links_output)
df.head()

FileNotFoundError: [Errno 2] File /media/kumar/Soft Disk/Developement_&_Programming/Python_Projects/amazon_price_tracking/notebooks/data/category-products.csv does not exist: '/media/kumar/Soft Disk/Developement_&_Programming/Python_Projects/amazon_price_tracking/notebooks/data/category-products.csv'

In [13]:
df.apply(row_scrape_event, axis=1)

NameError: name 'df' is not defined

In [15]:
df

NameError: name 'df' is not defined

In [ ]:
df.to_csv(product_output, index=False)
